In [7]:
import pandas as pd
from sqlalchemy import create_engine
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine
import platform
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tqdm import tqdm
import statsmodels.api as sm
import ssl
import certifi
from urllib.request import urlopen
import json
from DATA.stock_invest_function import *

In [44]:
def calculate_correlation_between_dfs(df1, df2, start_date=None, end_date=None, method='pearson', min_periods=4):
    """
    두 개의 시계열 DataFrame의 상관관계를 계산하되, 유효 관측치가 min_periods보다 많을 경우만 수행

    Parameters:
    ...
    - min_periods (int): 최소 유효 데이터 수

    Returns:
    - pd.DataFrame: 상관계수 매트릭스
    """
    # ✅ 인덱스를 datetime으로 변환
    df1.index = pd.to_datetime(df1.index)
    df2.index = pd.to_datetime(df2.index)

    if start_date:
        df1 = df1[df1.index >= pd.to_datetime(start_date)]
        df2 = df2[df2.index >= pd.to_datetime(start_date)]
    if end_date:
        df1 = df1[df1.index <= pd.to_datetime(end_date)]
        df2 = df2[df2.index <= pd.to_datetime(end_date)]

    combined = pd.merge(df1, df2, left_index=True, right_index=True, how='inner', suffixes=('_firm', '_hs'))

    corr_matrix = pd.DataFrame(index=df1.columns, columns=df2.columns, dtype=float)

    for firm in df1.columns:
        for hs in df2.columns:
            x = combined[firm]
            y = combined[hs]
            valid = x.notna() & y.notna()
            if valid.sum() >= min_periods:
                corr_matrix.loc[firm, hs] = x[valid].corr(y[valid], method=method)
            else:
                corr_matrix.loc[firm, hs] = np.nan  # 또는 0

    return corr_matrix

def get_top_correlated_hscode(corr_matrix, symbol, top_n=5, threshold=None, ascending=False):
    """
    특정 기업(Symbol)에 대해 상관관계가 높은 HS 코드를 추출하는 함수

    Parameters:
    - corr_matrix (pd.DataFrame): Symbol x HS_Code 형태의 상관관계 행렬
    - symbol (str): 대상 Symbol (예: '000080')
    - top_n (int): 상위 N개 추출 (threshold와 함께 사용 시 무시될 수 있음)
    - threshold (float or None): 상관계수 하한값 (예: 0.5 이상만 보기). 설정 시 top_n보다 우선함
    - ascending (bool): 상관계수 기준 오름차순 정렬 여부 (기본값: False = 높은 값 우선)

    Returns:
    - pd.DataFrame: root_hs_code 및 상관계수를 포함한 상위 N개 HS 코드
    """

    if symbol not in corr_matrix.index:
        raise ValueError(f"Symbol '{symbol}' not found in correlation matrix.")

    symbol_corr = corr_matrix.loc[symbol].dropna()

    if threshold is not None:
        symbol_corr = symbol_corr[symbol_corr >= threshold]

    top_correlated = symbol_corr.sort_values(ascending=ascending).head(top_n)

    return top_correlated.reset_index().rename(columns={'index': 'root_hs_code', symbol: 'correlation'})

def get_top_correlated_symbols(corr_matrix, hs_code, top_n=5, threshold=None, ascending=False):
    """
    특정 HS 코드에 대해 상관관계가 높은 기업 Symbol을 추출하는 함수

    Parameters:
    - corr_matrix (pd.DataFrame): Symbol x HS_Code 형태의 상관관계 행렬
    - hs_code (str or int): 대상 HS 코드 (예: '151550')
    - top_n (int): 상위 N개 추출
    - threshold (float or None): 상관계수 하한값 (예: 0.5 이상만 보기)
    - ascending (bool): 정렬 방향 (False: 높은 상관 우선)

    Returns:
    - pd.DataFrame: symbol 및 correlation 정보를 담은 상위 N개 결과
    """

    if hs_code not in corr_matrix.columns:
        raise ValueError(f"HS code '{hs_code}' not found in correlation matrix columns.")

    hs_corr = corr_matrix[hs_code].dropna()

    if threshold is not None:
        hs_corr = hs_corr[hs_corr >= threshold]

    top_symbols = hs_corr.sort_values(ascending=ascending).head(top_n)

    return top_symbols.reset_index().rename(columns={'index': 'symbol', hs_code: 'correlation'})

In [3]:
db_info = {
    'host': 'hystox74.synology.me',
    'port': 3307,
    'user': 'stox7412',
    'password': 'Apt106503!~',
    'database': 'investar'
}

# SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
)

# 테이블 이름
table_name = 'target_hs_code'

# 고유한 hs_code 값 추출 쿼리 실행
query = f"SELECT DISTINCT hs_code FROM {table_name}"
unique_hs_codes_df = pd.read_sql(query, con=engine)
hs_codes  = unique_hs_codes_df['hs_code'].unique().tolist()

indicator = 'expDlr'

df_real = fetch_trade_data_multi_hscode(db_info, hs_codes, indicator)

In [4]:
# 분기 정보 추가
df_real['quarter'] = df_real['date'].dt.to_period('Q')

# 그룹별로 분기별 합산
df_quarterly = (
    df_real
    .groupby(['root_hs_code', 'quarter'])['value']
    .sum()
    .reset_index()
)

# 👉 분기 월말로 변환 (예: 2007Q1 → 2007-03-31)
df_quarterly['date'] = df_quarterly['quarter'].dt.to_timestamp(how='end')

# 👉 'quarter' 컬럼 제거
df_quarterly.drop(columns=['quarter'], inplace=True)

# 1단계: 문자열로 직접 변환하려면 to_datetime 이후에 바로 strftime
df_quarterly['date'] = pd.to_datetime(df_quarterly['date']).dt.strftime('%Y-%m-%d')

def create_yoy_growth_pivot(df_quarterly, start_date=None, end_date=None):
    """
    전년 동분기 대비 증가율을 pivot 형태로 변환하고 분석기간을 설정할 수 있는 함수

    Parameters:
    - df_quarterly (DataFrame): 'root_hs_code', 'date', 'yoy_growth' 포함된 데이터
    - start_date (str or None): 분석 시작일 (예: '2015-01-01')
    - end_date (str or None): 분석 종료일 (예: '2023-12-31')

    Returns:
    - pivot_df (DataFrame): 행: date, 열: root_hs_code, 값: yoy_growth
    """
    # Pivot
    pivot_df = df_quarterly.pivot(
        index='date',
        columns='root_hs_code',
        values='yoy_growth'
    ).sort_index()

    # inf 값 NaN 처리
    pivot_df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # 분석 기간 슬라이싱 (날짜가 문자열이면 datetime으로 변환)
    pivot_df.index = pd.to_datetime(pivot_df.index)

    if start_date:
        pivot_df = pivot_df[pivot_df.index >= pd.to_datetime(start_date)]
    if end_date:
        pivot_df = pivot_df[pivot_df.index <= pd.to_datetime(end_date)]

    return pivot_df


# 전년 동분기 값 (4개 분기 전 값) 계산
df_quarterly['yoy_value'] = (
    df_quarterly
    .sort_values(['root_hs_code', 'date'])
    .groupby('root_hs_code')['value']
    .shift(4)
)

# ❗ yoy_growth 계산
df_quarterly['yoy_growth'] = (
    (df_quarterly['value'] - df_quarterly['yoy_value']) / df_quarterly['yoy_value']
) * 100

quarterly_trade_data = create_yoy_growth_pivot(df_quarterly, start_date='2008-03', end_date='2025-03')

In [5]:
quarterly_trade_data

root_hs_code,121120,121221,151550,151590,170199,190230,190590,200599,200830,200899,...,903149,903180,903190,903289,940130,940199,940330,940540,950300,970191
date,,,,,,,,,,,,,,,,,,,,,
2008-03-31,90.336099,NaN,16.777894,45.319785,-2.089371,19.901271,19.931377,6.957235,-2.343829,-33.535181,...,138.220984,41.192713,31.732529,80.695522,-47.403236,NaN,-9.986462,174.910924,-15.512682,NaN
2008-06-30,20.815636,NaN,112.829026,100.885760,-16.864182,18.872828,19.753492,15.020885,0.999482,30.527594,...,41.938920,36.543978,42.186509,-23.715247,53.141968,NaN,-18.646441,343.067034,-9.585525,NaN
2008-09-30,-41.073208,NaN,55.462621,33.656911,-10.472852,23.743338,7.138119,17.262148,37.208676,-15.271706,...,40.219235,-11.991021,36.523453,11.187714,48.790449,NaN,-47.857885,349.811942,-8.644475,NaN
2008-12-31,3.144400,NaN,-39.795990,144.057451,-0.056993,-8.459365,-3.608346,13.263258,6.142602,-23.742025,...,-30.760604,-8.942473,-9.105317,-38.998382,21.072238,NaN,5.974349,182.631540,-14.960084,NaN
2009-03-31,1.288444,NaN,115.673004,-9.411258,-21.750020,-5.503269,-7.306533,4.732786,-24.087312,-13.081807,...,-69.204685,-10.281097,-40.196447,-37.350827,-21.135143,NaN,-46.536388,-47.573484,-18.793910,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31,5.035949,21.451449,-3.635093,-8.826610,20.303902,27.255952,20.929892,11.717477,-9.394534,37.554516,...,-13.899575,25.187896,17.375068,45.880632,NaN,0.907928,15.475628,NaN,15.779052,-23.900927
2024-03-31,-38.869973,20.516295,-2.935003,-6.483635,33.624345,28.998818,3.954067,5.975202,-18.734050,16.652831,...,-28.634298,1.866472,26.750878,16.182839,NaN,4.233538,64.329712,NaN,0.568879,-29.012535
2024-06-30,40.483316,65.488901,44.078181,50.584872,-3.949636,32.371678,3.630900,1.988888,-16.947354,18.330731,...,19.021802,-2.056160,10.357521,14.610146,NaN,2.455197,56.423311,NaN,12.462183,68.391967


In [8]:
db_info = {
    'user': 'stox7412',         # 예: 'root'
    'password': 'Apt106503!~', # 예: '1234'
    'host': 'hystox74.synology.me',         # 예: 'localhost' 또는 IP
    'port': '3307',              # 기본 포트는 보통 3306
    'database': 'investar'        # 예: 'trade_data'
}

fs_df = fetch_table_data(db_info, "US_fundq")


✅ 'US_fundq' 테이블에서 1572228건의 데이터를 가져왔습니다.


In [17]:
# 0. 사전 준비
df = fs_df.copy()

# 1. 날짜 컬럼 정제
df['date'] = pd.to_datetime(df['date'])

# 2. 분기 종료일 기준으로 집계
df['quarter'] = df['date'].dt.to_period('Q').dt.to_timestamp(how='end')

# 3. 원하는 재무 변수만 선택
target_vars = ['saleq']
id_var = 'tic'  # 고유 기업 식별자
value_vars = [col for col in df.columns if col in target_vars]

# 4. 필요한 열만 추출하고 결측 제외
df_filtered = df[['quarter', id_var] + value_vars].dropna(subset=value_vars, how='all')

# 5. 긴 형태로 변환
df_long = df_filtered.melt(
    id_vars=['quarter', id_var],
    value_vars=value_vars,
    var_name='metric',
    value_name='value'
)

# 6. 피벗: (행: quarter, 열: (tic, metric), 값: value)
pivot_df = df_long.pivot_table(index='quarter', columns=[id_var, 'metric'], values='value')

# 7. 전년 동분기 대비 증가율 계산
yoy_growth = pivot_df.pct_change(periods=4) * 100

# 8. index를 YYYY-MM-DD 문자열로 변경
yoy_growth.index = yoy_growth.index.strftime('%Y-%m-%d')

# 9. 컬럼명이 MultiIndex일 경우, 'tic'만 남기기
if isinstance(yoy_growth.columns, pd.MultiIndex):
    yoy_growth.columns = yoy_growth.columns.get_level_values(0)

# 결과
fs_yoy_growth_df = yoy_growth

In [20]:
correlation_result = calculate_correlation_between_dfs(
    fs_yoy_growth_df,
    quarterly_trade_data,
    start_date='2015-03-31',
    end_date='2025-03-31'
)

# 상위 몇 개 확인
correlation_result.head()

root_hs_code,121120,121221,151550,151590,170199,190230,190590,200599,200830,200899,...,903149,903180,903190,903289,940130,940199,940330,940540,950300,970191
tic,,,,,,,,,,,,,,,,,,,,,
3ABTE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3ACEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3ACKH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3ACMI.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3ACTRQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# 모든 컬럼이 NaN인 행 제거
correlation_result_cleaned = correlation_result.dropna(how='all')
correlation_result_cleaned.to_excel('corelation_result_US.xlsx')

In [43]:
correlation_result_cleaned[['854232']]

root_hs_code,854232
tic,
3BCTI,-0.115108
3BHMSQ,-0.146344
3CAPS.,0.348480
3CHLD,-0.103990
3DAYR,-0.091834
...,...
ZVRA,NaN
ZWS,-0.404910
ZY,0.704715


In [47]:
top_hs_codes = get_top_correlated_hscode(
    corr_matrix=correlation_result,  # 이전에 만든 상관관계 행렬
    symbol='NVDA',
    top_n=20,
    threshold=0.3  # 선택사항
)

print(top_hs_codes)

   root_hs_code  correlation
0        852491     0.921992
1        854141     0.909876
2        852411     0.908073
3        852412     0.779539
4        851419     0.767473
5        700490     0.747236
6        850710     0.746234
7        852589     0.735844
8        854149     0.690194
9        900290     0.661028
10       847330     0.647964
11       840690     0.616507
12       390729     0.596655
13       732290     0.593482
14       854232     0.581137
15       940199     0.577874
16       392190     0.576307
17       845730     0.567425
18       721934     0.546201
19       382219     0.544172


In [48]:
top_symbols = get_top_correlated_symbols(
    corr_matrix=correlation_result,
    hs_code='847330',
    top_n=30,
    threshold=0.1  # 선택사항
)

print(top_symbols)

      tic  correlation
0    HPCO     0.946111
1     BVH     0.931274
2    FINW     0.928356
3     APP     0.927015
4   OSAPQ     0.904453
5    ROOT     0.877134
6     ISO     0.862219
7    MGRX     0.833020
8    CTCM     0.829972
9    CELU     0.805675
10    RMO     0.796665
11     MU     0.782706
12  GLSHQ     0.752770
13   SOUN     0.738783
14   BLZE     0.729357
15     ZY     0.724888
16    NXT     0.716174
17    GVA     0.706595
18  SHPWQ     0.705245
19   UTSI     0.680581
20   LNKB     0.648640
21   NVDA     0.647964
22   VEEA     0.647832
23   GOCO     0.647100
24    SHC     0.645618
25   BAER     0.637675
26    LSF     0.623956
27   COIN     0.616648
28   MOBQ     0.604811
29     EP     0.602674
